# Приступаем к работе  

Находим датасет и приступаем к предобработке

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import kagglehub

In [ ]:
# Download latest version
path = kagglehub.dataset_download("adilshamim8/people-detection")

print("Path to dataset files:", path)

In [ ]:
# Выводим содержимое
for item in os.listdir(path):
    print(item+":")
    if os.path.isdir(path+"/"+item):
        # Выводим содержимое
        for iitem in os.listdir(path+"/"+item):
            print("\t"+iitem+":")
            if os.path.isdir(path+"/"+item+"/"+iitem):
                # Выводим содержимое
                for iiitem in os.listdir(path+"/"+item+"/"+iitem):
                    print("\t"+"\t"+iiitem)

In [ ]:
# Функция для загрузки изображений из папки на OpenCV
def load_images(folder, target_size=(512, 512), allowed_extensions=("jpg", "jpeg", "png", "bmp", "gif")):
    images = []
    filenames = []

    if not os.path.exists(folder):
        raise FileNotFoundError(f"Папка {folder} не найдена.")
    
    # Отладочное сообщение
    print(f"Проверка файлов в папке {folder}...")

    # Перебираем все файлы в папке
    for filename in sorted(os.listdir(folder)):
        # Отладочное сообщение
        print(f"Проверяется файл: {filename}")
        
        if filename.lower().endswith(allowed_extensions):
            image_path = os.path.join(folder, filename)  # Полный путь к изображению
            print(f"Загружаем изображение: {image_path}")  # Отладочное сообщение
            
            try:
                # Загружаем изображение с OpenCV
                image = cv2.imread(image_path)
                if image is None:
                    print(f"Ошибка загрузки: {filename}")
                    continue
                
                # Изменяем размер изображения
                image_resized = cv2.resize(image, target_size)

                # Добавляем изображение в список
                images.append(image_resized)
                
                # Добавляем имя файла
                filenames.append(filename)
            except Exception as e:
                print(f"Ошибка при загрузке и обработке изображения {filename}: {e}")

    return images, filenames
                               

# Сохранение файла
# mh.imsave()

# Указываем путь к папке с изображениями
folder_path = "./data/images/teg"
images, filenames = load_images(folder_path)

# Проверяем, были ли загружены изображения
if not images:
    print("Нет изображений для отображения.")
else:
    # Ограничиваем количество изображений для отображения
    count_images_to_display = min(20, len(images))

    # Количество столбцов для отображения
    cols = 5
    rows = count_images_to_display // cols + (count_images_to_display % cols > 0)

# Настройка фигуры для отображения
    plt.figure(figsize=(15, 5 * rows))
    for i in range(count_images_to_display):
        plt.subplot(rows, cols, i + 1)
        # Отображаем изображение и Конвертируем BGR -> RGB
        plt.imshow(cv2.cvtColor(images[i], cv2.COLOR_BGR2RGB))
        plt.title(f"{filenames[i]}")    # Подпись с именем файла
        plt.axis('off') # Отключаем оси
    
    # Отображаем изображения
    plt.tight_layout() 
    plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):
    def __init__(self, input, output, is_downsample=False):
        super(BasicBlock, self).__init__()
        self.is_downsample = is_downsample
        if is_downsample:
            self.conv1 = nn.Conv2d(input, output, 3, stride=3, padding=1, bias=False)
        else:
            self.conv1 = nn.Conv2d(input, output, 3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(output)
        self.relu = nn.ReLU(True)
        self.conv2 = nn.Conv2d(output,output,3,stride=1,padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(output)
        if is_downsample:
            self.downsample = nn.Sequential(
                nn.Conv2d(input, output, 1, stride=2, bias=False),
                nn.BatchNorm2d(output)
            )
        elif input != output:
            self.downsample = nn.Sequential(
                nn.Conv2d(input, output, 1, stride=1, bias=False),
                nn.BatchNorm2d(output)
            )
            self.is_downsample = True

    def forward(self,x):
        y = self.conv1(x)
        y = self.bn1(y)
        y = self.relu(y)
        y = self.conv2(y)
        y = self.bn2(y)
        if self.is_downsample:
            x = self.downsample(x)
        return F.relu(x.add(y),True)

def make_layers(input,output,repeat_times, is_downsample=False):
    blocks = []
    for i in range(repeat_times):
        if i ==0:
            blocks += [BasicBlock(input,output, is_downsample=is_downsample),]
        else:
            blocks += [BasicBlock(output,output),]
    return nn.Sequential(*blocks)

class Net(nn.Module):
    def __init__(self, num_classes=625 ,reid=False):
        super(Net,self).__init__()
        # 3 128 64
        self.conv = nn.Sequential(
            nn.Conv2d(3,32,3,stride=1,padding=1),
            nn.BatchNorm2d(32),
            nn.ELU(inplace=True),
            nn.Conv2d(32,32,3,stride=1,padding=1),
            nn.BatchNorm2d(32),
            nn.ELU(inplace=True),
            nn.MaxPool2d(3,2,padding=1),
        )
        # 32 64 32
        self.layer1 = make_layers(32,32,2,False)
        # 32 64 32
        self.layer2 = make_layers(32,64,2,True)
        # 64 32 16
        self.layer3 = make_layers(64,128,2,True)
        # 128 16 8
        self.dense = nn.Sequential(
            nn.Dropout(p=0.6),
            nn.Linear(128*16*8, 128),
            nn.BatchNorm1d(128),
            nn.ELU(inplace=True)
        )
        # 256 1 1 
        self.reid = reid
        self.batch_norm = nn.BatchNorm1d(128)
        self.classifier = nn.Sequential(
            nn.Linear(128, num_classes),
        )
    
    def forward(self, x):
        x = self.conv(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = x.view(x.size(0),-1)
        if self.reid:
            x = self.dense[0](x)
            x = self.dense[1](x)
            x = x.div(x.norm(p=2,dim=1,keepdim=True))
            return x
        x = self.dense(x)
        # B x 128
        # classifier
        x = self.classifier(x)
        return x